In [457]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [449]:
#navigating files
raw_images_dir = 'images/raw_images/'
raw_images = os.listdir(raw_images_dir)
labeled_images_dir = 'images/labeled_images/'
labeled_images = os.listdir(labeled_images_dir)
metadata = 'game_titles.txt'

In [450]:
#alphabetize & remove duplicates from game title text file
def clean_metadata():
    with open(metadata, 'r') as file:
        items = []
        for line in file:
            items.extend(line.strip().split(', '))

    unique_items = sorted(set(item.strip() for item in items))

    with open(metadata, 'w') as file:
        file.write(', '.join(unique_items))

In [451]:
def build_dataframe():
    columns = ['title', 'year', 'perspective', 'genre', 'developer', 'publisher', 'raw_images', 'labeled_images']
    games_df = pd.DataFrame(columns=columns)
    
    with open(metadata, 'r') as file:
        titles = []
        years = []
        perspectives = []
        genres = []
        developers = []
        publishers = []

        for line in file:
            entries = line.strip().split(', ')
            for entry in entries:
                if '|' in entry:
                    # Split and fill missing values with defaults
                    split_entry = list(map(str.strip, entry.split('|')))
                    while len(split_entry) < 6:  # Ensure the split entry has six parts
                        split_entry.append("")  # Fill missing parts with empty strings
                    
                    # Unpack the entry
                    title, year, perspective, genre, developer, publisher = split_entry
                    
                    # Append to the respective lists
                    titles.append(title)
                    years.append(int(year) if year.isdigit() else 0)  # Convert year if numeric, otherwise 0
                    perspectives.append(perspective)
                    genres.append(genre)
                    developers.append(developer)
                    publishers.append(publisher)
                else:
                    # For entries without '|'
                    titles.append(entry.strip())
                    years.append(0)
                    perspectives.append("")
                    genres.append("")
                    developers.append("")
                    publishers.append("")

    # Compile the data into a DataFrame
    data = {
        'title': titles,
        'year': years,
        'perspective': perspectives,
        'genre': genres,
        'developer': developers,
        'publisher': publishers,
        'raw_images': [[] for _ in titles],
        'labeled_images': [[] for _ in titles]
    }

    games_df = pd.DataFrame(data)

    # Process raw_images and labeled_images
    for title in titles:
        for file in raw_images:
            if title == file[:-7]:
                games_df.loc[games_df['title'] == title, 'raw_images'] = (
                    games_df.loc[games_df['title'] == title, 'raw_images'].apply(lambda x: x + [raw_images_dir + file])
                )
    
    for title in titles:
        for file in labeled_images:
            if title == file[:-7]:
                games_df.loc[games_df['title'] == title, 'labeled_images'] = (
                    games_df.loc[games_df['title'] == title, 'labeled_images'].apply(lambda x: x + [labeled_images_dir + file])
                )
    
    return games_df


In [452]:
games_df = build_dataframe()

In [453]:
def filter_games(criteria):
    filtered_games = games_df[
        games_df['title'].isin(criteria) |
        games_df['year'].isin(criteria) |
        games_df['perspective'].isin(criteria) |
        games_df['genre'].isin(criteria) |
        games_df['developer'].isin(criteria) |
        games_df['publisher'].isin(criteria)
    ]
    return(filtered_games)

In [454]:
games_df

,title,year,perspective,genre,developer,publisher,raw_images,labeled_images
0,Alien Isolation,2014,first-person,survival horror,Creative Assembly,Sega,[images/raw_images/Alien Isolation 01.jpg],[images/labeled_images/Alien Isolation 01.xml]
1,Back 4 Blood,2021,first-person,first-person shooter,Turtle Rock Studios,Warner Bros. Games,[images/raw_images/Back 4 Blood 01.jpg],[images/labeled_images/Back 4 Blood 01.xml]
2,Battlefield 1,2016,first-person,first-person shooter,DICE,Electronic Arts,[images/raw_images/Battlefield 1 01.jpg],[images/labeled_images/Battlefield 1 01.xml]
3,Battlefield 2042,2021,first-person,first-person shooter,DICE,Electronic Arts,[images/raw_images/Battlefield 2042 01.jpg],[images/labeled_images/Battlefield 2042 01.xml]
4,Battlefield V,2018,first-person,first-person shooter,DICE,Electronic Arts,[images/raw_images/Battlefield V 01.jpg],[images/labeled_images/Battlefield V 01.xml]
...,...,...,...,...,...,...,...,...
59,The Outer Worlds,2019,first-person,first-person shooter,Obsidian Entertainment,Private Division,[images/raw_images/The Outer Worlds 01.jpg],[]
60,Tom Clancys Rainbow Six Siege,2015,first-person,tactical-shooter,Ubisoft Montreal,Ubisoft,[images/raw_images/Tom Clancys Rainbow Six Sie...,[]
61,Wolfenstein II The New Colossus,0,first-person shooter,MachineGames,Bethesda Softworks,,[images/raw_images/Wolfenstein II The New Colo...,[]
62,Wolfenstein The New Order,2014,first-person,first-person shooter,MachineGames,Bethesda Softworks,[images/raw_images/Wolfenstein The New Order 0...,[]
